#**Import libraries**

In [ ]:
# For data loading and preprocessing
import pandas as pd

# For Text preprocessing
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# For the model and training
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np

# For Prediction and accuracy check
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#**Load Data**

In [ ]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


#**Data Preprocessing**

In [ ]:
# Drop NaN Values
df=df.dropna()

# Get the Independent Features
X = df.drop('label', axis=1)

# Get the Dependent features
y = df['label']

# Since some rows are dropped for NaN values, hence we need to reset the indices
messages = X.copy()
messages.reset_index(inplace=True)

#**Text Preprocessing + Encoding + Embedding + Model creation**

In [ ]:
# To remove very frequently used words
nltk.download('stopwords')

# Object to apply stemming
ps = PorterStemmer()

corpus = []
for i in range(0, len(messages)):
  # Using regular expressions (regex) to keep only alphabets and remove all other special characters
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])

  # Lower casing all alphabets
  review = review.lower()

  # Splitting all the strings based on spaces
  review = review.split()

  # Removing stopwords + application of stemming to the words
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)

  # Appending final stemmed sentences to our corpus (the array which will be used for vocabulary creation and then embedding)
  corpus.append(review)

#print(corpus)

# Forming the vocabulary
vocabulary_size = 10000
onehot_repr=[one_hot(words, vocabulary_size) for words in corpus]

# Padding zeroes to the start of sentences so that all sentences are of equal length
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)

# Embedding
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_vector_features, input_length = sent_length))

# 1 LSTM layer with 100 neurons
model.add(LSTM(100))

# Output/final layer
model.add(Dense(1,activation='sigmoid'))

# Compiling model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            400000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 456501 (1.74 MB)
Trainable params: 456501 (1.74 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


#**Preparing Data**

In [ ]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state = 42)

#**Model Training**

In [ ]:
model.fit(X_train, y_train, validation_split = 0.33, epochs=100, batch_size = 32)

Epoch 1/100
257/257 [==============================] - 7s 28ms/step - loss: 0.0087 - accuracy: 0.9974 - val_loss: 0.4297 - val_accuracy: 0.8979
Epoch 2/100
257/257 [==============================] - 8s 33ms/step - loss: 0.0127 - accuracy: 0.9971 - val_loss: 0.4815 - val_accuracy: 0.9107
Epoch 3/100
257/257 [==============================] - 9s 34ms/step - loss: 0.0041 - accuracy: 0.9987 - val_loss: 0.5809 - val_accuracy: 0.9158
Epoch 4/100
257/257 [==============================] - 9s 34ms/step - loss: 0.0016 - accuracy: 0.9995 - val_loss: 0.5962 - val_accuracy: 0.9128
Epoch 5/100
257/257 [==============================] - 8s 32ms/step - loss: 8.9551e-04 - accuracy: 0.9998 - val_loss: 0.6385 - val_accuracy: 0.9128
Epoch 6/100
257/257 [==============================] - 8s 32ms/step - loss: 2.0363e-04 - accuracy: 1.0000 - val_loss: 0.6570 - val_accuracy: 0.9097
Epoch 7/100
257/257 [==============================] - 8s 32ms/step - loss: 1.2972e-04 - accuracy: 1.0000 - val_loss: 0.7025 - v

#**Prediction and accuracy**

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
accuracy_score(y_test, y_pred)

189/189 [==============================] - 2s 8ms/step


0.9085335542667772